In [ ]:
import pandas as pd

path = '../data/eniac/'
orderlines = pd.read_csv(path + 'orderlines.csv')
orders = pd.read_csv(path + 'orders.csv')

def start_pipeline(data): 
    return data.copy()

def set_types_orders(data): 
    return data.assign(create_date = pd.to_datetime(data['created_date']))

def common_orders(orders, orderlines): 
    return (orders
            .assign(order_check = orders['order_id'].isin(orderlines['id_order']))
            .query('order_check == True')
            ['order_id'].tolist())

def select_common_orders(data, orders_list: list): 
    if 'order_id' in data.columns: 
        data.rename(columns={'order_id':'id_order'}, inplace=True)
    data_filtered = data.query('id_order == @orders_list')
    print(
        f"""
        Total {data_filtered.shape[0]} rows included over the original {data.shape[0]} rows.\n
        Kept the {round((data_filtered.shape[0] / data.shape[0])*100, 2)}% of the data.
        """)
    return data_filtered    

(
orders
    .pipe(start_pipeline)
    .pipe(set_types_orders)
    .dropna()
    .pipe(select_common_orders, orders_list=common_orders(orders, orderlines))
)